In [1]:
from ultralytics import YOLO
from roboflow import Roboflow

# 1. データの準備
rf = Roboflow(api_key="OJhli9tA7VXWzXHAgQuj")
project = rf.workspace("g11-temb0").project("-8sx96")
version = project.version(4)
dataset = version.download("yolov11")

# 2. 学習の再実行（エラー回避設定済み）
model = YOLO("yolo11n.pt")

results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=50,
    imgsz=640,
    plots=False  # <--- 重要！これでグラフ描画エラーを回避して保存を優先します
)

# 3. 学習が終わると、保存場所が表示されます
# 例: "Results saved to runs/detect/train"
print("学習完了！")

loading Roboflow workspace...
loading Roboflow project...
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.1 CPU (Apple M3)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/正答検知-4/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train6, nbs=64, nms=False, opset=None, optimiz

In [5]:
import os
import glob
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

def find_latest_model():
    # runs/detect/train* の中で一番新しいフォルダを探す
    search_dir = "runs/detect/"
    dirs = glob.glob(os.path.join(search_dir, "train*"))
    if not dirs:
        return None
    # 作成日時順に並び替え
    latest_dir = max(dirs, key=os.path.getmtime)
    return os.path.join(latest_dir, "weights", "best.pt")

def detect_four_marks_auto(image_path):
    # 最新のモデルパスを自動取得
    model_path = find_latest_model()
    
    if model_path is None or not os.path.exists(model_path):
        print("エラー: 学習済みモデルが見つかりません。手順1の学習を実行してください。")
        return

    print(f"使用するモデル: {model_path}")
    model = YOLO(model_path)
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"画像が見つかりません: {image_path}")
        return

    results = model(img)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            label = model.names[cls]

            if conf < 0.4: continue # 信頼度

            # ラベルごとの色設定（必要に応じて書き換えてください）
            if "maru" in label or "circle" in label:
                color = (255, 0, 0) # 青
            elif "batsu" in label or "cross" in label:
                color = (0, 0, 255) # 赤
            elif "check" in label:
                color = (0, 255, 0) # 緑
            elif "sankaku" in label or "triangle" in label:
                color = (0, 255, 255) # 黄
            else:
                color = (128, 128, 128)

            cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
            cv2.putText(img, f"{label} {conf:.2f}", (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# --- 実行 ---
# 画像パスだけ指定すればOKです
image_file = "/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/sample.png"
detect_four_marks_auto(image_file)

エラー: 学習済みモデルが見つかりません。手順1の学習を実行してください。


In [ ]:
import os
import glob
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

def find_latest_model():
    # runs/detect/train* の中で一番新しいフォルダを探す
    search_dir = ""
    dirs = glob.glob(os.path.join(search_dir, "train*"))
    if not dirs:
        return None
    # 作成日時順に並び替え
    latest_dir = max(dirs, key=os.path.getmtime)
    return os.path.join(latest_dir, "weights", "best.pt")

def detect_four_marks_auto(image_path):
    # 最新のモデルパスを自動取得
    model_path = find_latest_model()
    
    if model_path is None or not os.path.exists(model_path):
        print("エラー: 学習済みモデルが見つかりません。手順1の学習を実行してください。")
        return

    print(f"使用するモデル: {model_path}")
    model = YOLO(model_path)
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"画像が見つかりません: {image_path}")
        return

    results = model(img)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])
            cls = int(box.cls[0])
            label = model.names[cls]

            if conf < 0.4: continue # 信頼度

            # ラベルごとの色設定（必要に応じて書き換えてください）
            if "maru" in label or "circle" in label:
                color = (255, 0, 0) # 青
            elif "batsu" in label or "cross" in label:
                color = (0, 0, 255) # 赤
            elif "check" in label:
                color = (0, 255, 0) # 緑
            elif "sankaku" in label or "triangle" in label:
                color = (0, 255, 255) # 黄
            else:
                color = (128, 128, 128)

            cv2.rectangle(img, (x1, y1), (x2, y2), color, 3)
            cv2.putText(img, f"{label} {conf:.2f}", (x1, y1 - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

# --- 実行 ---
# 画像パスだけ指定すればOKです
image_file = "/Users/mihiraryouta/ハッカソン/学内ハッカソン/asobitai/smart_doc_gen/services/sample.png"
detect_four_marks_auto(image_file)

エラー: 学習済みモデルが見つかりません。手順1の学習を実行してください。
